In [56]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("mySQLite") \
    .config("spark.jars.packages", "org.xerial:sqlite-jdbc:3.46.0.0") \
    .getOrCreate()

In [57]:
# Define URL JDBC và các thuộc tính kết nối
jdbc_url = "jdbc:sqlite:/myProject/yelp.db"  # URL JDBC cho cơ sở dữ liệu SQLite
connection_properties = {
    "driver": "org.sqlite.JDBC"  # Chỉ định lớp driver cho SQLite
}

df = spark.read.jdbc(url=jdbc_url, table="yelp", properties=connection_properties)

In [58]:
#Turn off WARNING
import logging
from pyspark.sql import SparkSession
logging.getLogger('py4j').setLevel(logging.ERROR)
spark.sparkContext.setLogLevel("ERROR")


In [59]:
df.show()

+-----------+--------------------+------+------------+--------+-----+--------------+--------------------+--------+
|      Phone|        Organization|Rating|NumberReview|Category|State|          City|              Street|Building|
+-----------+--------------------+------+------------+--------+-----+--------------+--------------------+--------+
|12562343500|  The Station's Best|   4.0|           4|Delivery|   AL|Alexander City|           Jefferson|     977|
|12566758004|           Pizza Hut|   3.0|           6|Delivery|   AL|Alexander City|            4581 Hwy|    4581|
|12562342181|Zaxby's Chicken F...|   3.0|          13|Delivery|   AL|Alexander City|        4497 Highway|    4497|
|12564097272|   Papa John's Pizza|   1.0|           1|Delivery|   AL|Alexander City|       2064 Cherokee|    2064|
|12562155510|              Arby's|   2.0|           7|Delivery|   AL|Alexander City|            2593 Hwy|    2593|
|12562343500|  The Station's Best|   4.0|           4|Delivery|   AL|Alexander C

In [60]:
df.count()

1000000

Create operation

In [61]:
from pyspark.sql import Row

# Tạo một hàng dữ liệu mới
new_row = Row(Phone = 1284680203, Organization = 'ChickenGarden', Rating = 3.0, NumberReview = 100, Category = 'Chicken', State = 'AL', City = 'Oxford', Street = 'Beclin', Building = '131' )
# Tạo DataFrame mới (new_df) chỉ có new_row
new_df = spark.createDataFrame([new_row], schema=df.schema)

df = df.union(new_df)

df.tail(3)
df.count()


1000001

Delete Operation

In [62]:
df = df.filter(~((df.Phone == 13342221398) & 
                 (df.Organization == 'Happy Kitchen') & 
                 (df.Rating == 3.5) & 
                 (df.NumberReview == 11) & 
                 (df.Category == 'Delivery') & 
                 (df.State == 'AL') & 
                 (df.City == 'Andalusia') & 
                 (df.Street == ' 700 Western Gate') & 
                 (df.Building == '700')))

print(df.count())
df.head(12)
#Row 12

999999


[Row(Phone=12562343500, Organization="The Station's Best", Rating=4.0, NumberReview=4, Category='Delivery', State='AL', City='Alexander City', Street=' Jefferson', Building='977'),
 Row(Phone=12566758004, Organization='Pizza Hut', Rating=3.0, NumberReview=6, Category='Delivery', State='AL', City='Alexander City', Street=' 4581 Hwy', Building='4581'),
 Row(Phone=12562342181, Organization="Zaxby's Chicken Fingers & Buffalo Wings", Rating=3.0, NumberReview=13, Category='Delivery', State='AL', City='Alexander City', Street=' 4497 Highway', Building='4497'),
 Row(Phone=12564097272, Organization="Papa John's Pizza", Rating=1.0, NumberReview=1, Category='Delivery', State='AL', City='Alexander City', Street=' 2064 Cherokee', Building='2064'),
 Row(Phone=12562155510, Organization="Arby's", Rating=2.0, NumberReview=7, Category='Delivery', State='AL', City='Alexander City', Street=' 2593 Hwy', Building='2593'),
 Row(Phone=12562343500, Organization="The Station's Best", Rating=4.0, NumberReview=4,

Update operation

In [63]:
from pyspark.sql.functions import when

# Update the DataFrame to change the NumberReview columns
df = df.withColumn('NumberReview', when((df.Organization == 'Pizza Hut') & (df.City == 'Alexander City') & (df.Rating == 3), 7).otherwise(df.NumberReview))

df.head(10)


[Row(Phone=12562343500, Organization="The Station's Best", Rating=4.0, NumberReview=4, Category='Delivery', State='AL', City='Alexander City', Street=' Jefferson', Building='977'),
 Row(Phone=12566758004, Organization='Pizza Hut', Rating=3.0, NumberReview=7, Category='Delivery', State='AL', City='Alexander City', Street=' 4581 Hwy', Building='4581'),
 Row(Phone=12562342181, Organization="Zaxby's Chicken Fingers & Buffalo Wings", Rating=3.0, NumberReview=13, Category='Delivery', State='AL', City='Alexander City', Street=' 4497 Highway', Building='4497'),
 Row(Phone=12564097272, Organization="Papa John's Pizza", Rating=1.0, NumberReview=1, Category='Delivery', State='AL', City='Alexander City', Street=' 2064 Cherokee', Building='2064'),
 Row(Phone=12562155510, Organization="Arby's", Rating=2.0, NumberReview=7, Category='Delivery', State='AL', City='Alexander City', Street=' 2593 Hwy', Building='2593'),
 Row(Phone=12562343500, Organization="The Station's Best", Rating=4.0, NumberReview=4,

Read operation

In [64]:
filtered_df = df.filter(df.Rating > 4)

# Show the filtered DataFrame
filtered_df.show()
print(filtered_df.count())

+-----------+--------------------+------+------------+--------+-----+------------+------------------+--------+
|      Phone|        Organization|Rating|NumberReview|Category|State|        City|            Street|Building|
+-----------+--------------------+------+------------+--------+-----+------------+------------------+--------+
|12568483131|       Marco's Pizza|   4.5|           3|Delivery|   AL|    Anniston|   2485 US Hwy 431|    2485|
|12563652001|        Jimmy John's|   4.5|           2|Delivery|   AL|Jacksonville|     505 Pelham Rd|     505|
|12564357272|   Papa John's Pizza|   4.5|           2|Delivery|   AL|Jacksonville|  702 South Pelham|     702|
|12564444005| Ramona Js Resturant|   5.0|           6|Delivery|   AL|      Athens|    1212 US Hwy 31|    1212|
|12562323000|   Papa John's Pizza|   4.5|           3|Delivery|   AL|      Athens| 916 Us Highway 72|     916|
|12562161099|       Papa Murphy's|   5.0|           1|Delivery|   AL|      Athens|       1001 Hwy 72|    1001|
|

The performance of some queries

In [65]:
%%time
df.createOrReplaceTempView("yelp")
spark.sql("SELECT * FROM yelp").show(5)

+-----------+--------------------+------+------------+--------+-----+--------------+--------------+--------+
|      Phone|        Organization|Rating|NumberReview|Category|State|          City|        Street|Building|
+-----------+--------------------+------+------------+--------+-----+--------------+--------------+--------+
|12562343500|  The Station's Best|   4.0|           4|Delivery|   AL|Alexander City|     Jefferson|     977|
|12566758004|           Pizza Hut|   3.0|           7|Delivery|   AL|Alexander City|      4581 Hwy|    4581|
|12562342181|Zaxby's Chicken F...|   3.0|          13|Delivery|   AL|Alexander City|  4497 Highway|    4497|
|12564097272|   Papa John's Pizza|   1.0|           1|Delivery|   AL|Alexander City| 2064 Cherokee|    2064|
|12562155510|              Arby's|   2.0|           7|Delivery|   AL|Alexander City|      2593 Hwy|    2593|
+-----------+--------------------+------+------------+--------+-----+--------------+--------------+--------+
only showing top 5 

In [66]:
%%time
spark.sql("SELECT * FROM yelp WHERE Rating > 4").show(5)

+-----------+-------------------+------+------------+--------+-----+------------+------------------+--------+
|      Phone|       Organization|Rating|NumberReview|Category|State|        City|            Street|Building|
+-----------+-------------------+------+------------+--------+-----+------------+------------------+--------+
|12568483131|      Marco's Pizza|   4.5|           3|Delivery|   AL|    Anniston|   2485 US Hwy 431|    2485|
|12563652001|       Jimmy John's|   4.5|           2|Delivery|   AL|Jacksonville|     505 Pelham Rd|     505|
|12564357272|  Papa John's Pizza|   4.5|           2|Delivery|   AL|Jacksonville|  702 South Pelham|     702|
|12564444005|Ramona Js Resturant|   5.0|           6|Delivery|   AL|      Athens|    1212 US Hwy 31|    1212|
|12562323000|  Papa John's Pizza|   4.5|           3|Delivery|   AL|      Athens| 916 Us Highway 72|     916|
+-----------+-------------------+------+------------+--------+-----+------------+------------------+--------+
only showi

In [69]:
%%time
spark.sql("SELECT City, SUM(NumberReview) as TotalNumberReview FROM yelp GROUP BY City").show()


+----------------+-----------------+
|            City|TotalNumberReview|
+----------------+-----------------+
|         Jemison|               25|
|      Prattville|             1093|
|       Fairbanks|             2334|
| Rancho Bernardo|              177|
|     Santa Paula|             1322|
|          Agawam|             1625|
|         Kilauea|              180|
|         Antwerp|                7|
|         Minster|               27|
|        Bluffton|              380|
|         Hanover|             9924|
|          Grimes|              672|
|        Fredonia|              264|
|          Nahant|              600|
|       Worcester|            12847|
|North Saint Paul|               14|
|         Palermo|               56|
|      Westampton|              618|
|  West Sand Lake|              104|
|       Rhinebeck|              640|
+----------------+-----------------+
only showing top 20 rows

CPU times: user 6.51 ms, sys: 2.36 ms, total: 8.87 ms
Wall time: 2.84 s


In [67]:
%%time
spark.sql("SELECT City, SUM(NumberReview) as TotalNumberReview FROM yelp WHERE Rating > 4 GROUP BY City").show()

# Hiển thị tổng số lượt đánh giá (TotalNumberReview) cho mỗi City có Rating lớn hơn 4.

+------------+-----------------+
|        City|TotalNumberReview|
+------------+-----------------+
|  Prattville|              155|
|   Fairbanks|              588|
| Santa Paula|              524|
|      Agawam|              616|
|     Kilauea|              167|
|     Minster|                6|
|      Grimes|               32|
|     Hanover|              814|
|   Worcester|             1968|
|     Palermo|               37|
|    Fredonia|               89|
|   Rhinebeck|              306|
| Johnsonburg|               30|
|       Tyler|              200|
|Saint George|              303|
|     Jemison|               11|
|  Hanceville|                4|
|    Moreland|                7|
|  Santee Cal|                9|
|    Bluffton|               28|
+------------+-----------------+
only showing top 20 rows

CPU times: user 5.12 ms, sys: 1.85 ms, total: 6.97 ms
Wall time: 1.56 s


In [68]:
%%time
spark.sql("SELECT City, SUM(NumberReview) as TotalNumberReview FROM yelp WHERE Rating > 4 GROUP BY City HAVING TotalNumberReview > 100").show()

# Thực hiện câu lệnh SQL để tính tổng giá trị (SUM(NumberReview)) của các hàng có Rating lớn hơn 4
# Nhóm kết quả theo City, và chỉ hiển thị các nhóm có SUM(NumberReview) lớn hơn 100.

+-------------+-----------------+
|         City|TotalNumberReview|
+-------------+-----------------+
|   Prattville|              155|
|    Fairbanks|              588|
|  Santa Paula|              524|
|       Agawam|              616|
|      Kilauea|              167|
|      Hanover|              814|
|    Worcester|             1968|
|    Rhinebeck|              306|
|        Tyler|              200|
| Saint George|              303|
|       Nahant|              420|
|        Tempe|            50837|
|       Corona|            52141|
|     Leucadia|              544|
|  Marcus Hook|              110|
|  Springfield|            12637|
|   Charleston|            11240|
|Bowling Green|              827|
|      Truckee|             2118|
|     Oakhurst|              120|
+-------------+-----------------+
only showing top 20 rows

CPU times: user 9.06 ms, sys: 0 ns, total: 9.06 ms
Wall time: 1.34 s
